In [ ]:
!pip install xgboost statsmodels matplotlib seaborn plotly mlflow boto3
# print("Required packages installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import logging
import boto3
from io import StringIO
import os
import pandas as pd
import numpy as np
from datetime import datetime
import mlflow
from mlflow.tracking import MlflowClient
# Configure prettier plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("muted")
sns.set_context("talk")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Suppress warnings for cleaner notebook output
warnings.filterwarnings("ignore")

print("Libraries imported successfully")
print("Environment setup complete")

Libraries imported successfully
Environment setup complete


In [ ]:
class S3Connector:
    """Utility class for connecting to S3 and retrieving files."""

    def __init__(self,
                 aws_access_key_id=None,
                 aws_secret_access_key=None,
                 region_name='us-east-1'):
        """
        Initialize S3 client.

        Args:
            aws_access_key_id (str, optional): AWS access key ID
            aws_secret_access_key (str, optional): AWS secret access key
            region_name (str, optional): AWS region name
        """
        self.s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            region_name=region_name
        )
        self.region_name = region_name
        print(f"✅ S3 client initialized for region: {region_name}")

    def read_csv_from_s3(self, bucket_name, key):
        """
        Read CSV file from S3 bucket.

        Args:
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)

        Returns:
            pandas.DataFrame: DataFrame containing CSV data
        """
        try:
            print(f"Reading file from S3: s3://{bucket_name}/{key}")
            response = self.s3_client.get_object(Bucket=bucket_name, Key=key)
            csv_content = response['Body'].read().decode('utf-8')
            df = pd.read_csv(StringIO(csv_content))
            print(f"✅ Successfully read CSV from S3 with {len(df):,} rows")
            return df
        except Exception as e:
            print(f"❌ Error reading file from S3: {str(e)}")
            return None

    def save_file_to_s3(self, local_file_path, bucket_name, key):
        """
        Save a local file to S3 bucket.

        Args:
            local_file_path (str): Local file path
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            print(f"Uploading file to S3: s3://{bucket_name}/{key}")
            self.s3_client.upload_file(local_file_path, bucket_name, key)
            print(f"✅ Successfully uploaded file to S3")
            return True
        except Exception as e:
            print(f"❌ Error uploading file to S3: {str(e)}")
            return False

    def save_data_to_s3(self, data, bucket_name, key, content_type=None):
        """
        Save data directly to S3 without creating a local file first.

        Args:
            data: Data to save (bytes, string, or serializable object)
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)
            content_type (str, optional): Content type of the data

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            print(f"Saving data to S3: s3://{bucket_name}/{key}")

            # Handle different data types
            if isinstance(data, bytes):
                body = data
            elif isinstance(data, str):
                body = data.encode('utf-8')
            else:
                # For other objects like dictionaries, try to JSON serialize
                try:
                    body = json.dumps(data).encode('utf-8')
                    if not content_type:
                        content_type = 'application/json'
                except:
                    print(f"❌ Error: Data type not supported for direct S3 upload")
                    return False

            # Set up the upload parameters
            params = {
                'Bucket': bucket_name,
                'Key': key,
                'Body': body
            }

            if content_type:
                params['ContentType'] = content_type

            # Upload the data
            self.s3_client.put_object(**params)
            print(f"✅ Successfully saved data to S3")
            return True

        except Exception as e:
            print(f"❌ Error saving data to S3: {str(e)}")
            return False

    def save_pickle_to_s3(self, obj, bucket_name, key):
        """
        Pickle an object and save it directly to S3.

        Args:
            obj: Python object to pickle
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            print(f"Saving pickled object to S3: s3://{bucket_name}/{key}")

            # Pickle the object to a bytes buffer
            import pickle
            from io import BytesIO

            buffer = BytesIO()
            pickle.dump(obj, buffer)
            buffer.seek(0)

            # Upload the pickled object
            self.s3_client.upload_fileobj(buffer, bucket_name, key)
            print(f"✅ Successfully saved pickled object to S3")
            return True

        except Exception as e:
            print(f"❌ Error pickling object to S3: {str(e)}")
            return False

    def save_figure_to_s3(self, figure, bucket_name, key, dpi=300, format='png'):
        """
        Save a matplotlib figure directly to S3.

        Args:
            figure: Matplotlib figure object
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)
            dpi (int): DPI for the figure
            format (str): Format to save the figure ('png', 'jpg', etc.)

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            print(f"Saving figure to S3: s3://{bucket_name}/{key}")

            # Save figure to a bytes buffer
            from io import BytesIO
            buffer = BytesIO()
            figure.savefig(buffer, format=format, dpi=dpi)
            buffer.seek(0)

            # Set appropriate content type based on format
            content_type = f"image/{format}"

            # Upload the figure
            self.s3_client.upload_fileobj(
                buffer,
                bucket_name,
                key,
                ExtraArgs={'ContentType': content_type}
            )
            print(f"✅ Successfully saved figure to S3")
            return True

        except Exception as e:
            print(f"❌ Error saving figure to S3: {str(e)}")
            return False

    def check_s3_path_exists(self, bucket_name, prefix):
        """
        Check if an S3 path (prefix) exists.

        Args:
            bucket_name (str): S3 bucket name
            prefix (str): Path prefix to check

        Returns:
            bool: True if path exists, False otherwise
        """
        try:
            # Ensure the prefix ends with a slash if it's meant to be a directory
            if prefix and not prefix.endswith('/'):
                prefix += '/'

            response = self.s3_client.list_objects_v2(
                Bucket=bucket_name,
                Prefix=prefix,
                MaxKeys=1
            )

            return 'Contents' in response
        except Exception as e:
            print(f"❌ Error checking S3 path: {str(e)}")
            return False

    def create_s3_directory(self, bucket_name, directory):
        """
        Create a directory structure in S3 (by adding an empty object with trailing slash).

        Args:
            bucket_name (str): S3 bucket name
            directory (str): Directory path to create

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            # Ensure the directory path ends with a slash
            if not directory.endswith('/'):
                directory += '/'

            # Create an empty object with the directory name (S3 convention for directories)
            self.s3_client.put_object(
                Bucket=bucket_name,
                Key=directory,
                Body=''
            )

            print(f"✅ Created S3 directory: s3://{bucket_name}/{directory}")
            return True
        except Exception as e:
            print(f"❌ Error creating S3 directory: {str(e)}")
            return False

In [ ]:
class DataUtils:
    """Utility class for data loading and preprocessing operations."""

    @staticmethod
    def load_data(s3_connector, bucket_name, key):
        """
        Load data from S3 bucket.

        Args:
            s3_connector (S3Connector): S3 connector instance
            bucket_name (str): S3 bucket name
            key (str): Object key (file path in bucket)

        Returns:
            pandas.DataFrame: Loaded data
        """
        print(f"Loading data from S3: s3://{bucket_name}/{key}")
        data = s3_connector.read_csv_from_s3(bucket_name, key)

        if data is None:
            print(f"❌ Error loading data from S3")
            return None

        print(f"✅ Successfully loaded {len(data):,} records with {len(data.columns)} columns")

        # Display sample data
        print("\n📊 Data Preview:")
        print(data.head())

        # Display data info
        print("\n📋 Data Information:")
        print(f"Shape: {data.shape}")
        print(f"Columns: {', '.join(data.columns)}")
        print(f"Date range: {pd.to_datetime(data['date']).min()} to {pd.to_datetime(data['date']).max()}")

        # Check for missing values
        missing_values = data.isnull().sum().sum()
        print(f"Missing values: {missing_values:,} ({missing_values/data.size:.2%} of all data)")

        return data

    @staticmethod
    def create_output_dirs(output_dir):
        """
        Create output directories for models and plots.

        Args:
            output_dir (str): Base output directory

        Returns:
            tuple: Paths to model registry and plots directories
        """
        print(f"Creating output directories in: {output_dir}")

        # Create main output directory
        os.makedirs(output_dir, exist_ok=True)

        # Create subdirectories
        model_registry_dir = os.path.join(output_dir, "model_registry")
        plots_dir = os.path.join(output_dir, "plots")

        os.makedirs(model_registry_dir, exist_ok=True)
        os.makedirs(plots_dir, exist_ok=True)

        print(f"✅ Created directory structure:")
        print(f"  - 📁 {output_dir} (main)")
        print(f"  - 📁 {model_registry_dir} (models)")
        print(f"  - 📁 {plots_dir} (visualizations)")

        return model_registry_dir, plots_dir


In [ ]:
class DataPreprocessor:
    """Class for preprocessing wheat price data."""

    def __init__(self):
        """Initialize the preprocessor."""
        self.date_col = None

    def preprocess_retail_prices(self, raw_data):
        """
        Preprocess retail price data.

        Args:
            raw_data (pandas.DataFrame): Raw input data

        Returns:
            pandas.DataFrame: Preprocessed retail data
        """
        print("Starting data preprocessing...")

        # Convert MSP from quintal to KG
        if 'MSP_Wheat' in raw_data.columns:
            raw_data['MSP_Wheat_KG'] = raw_data['MSP_Wheat'] / 100
            print("Converted MSP from quintal to KG")
        else:
            raw_data['MSP_Wheat_KG'] = 0
            print("MSP_Wheat column not found, using 0 as default")

        # Keep only retail prices if price type available
        if 'pricetype' in raw_data.columns:
            df_retail = raw_data[raw_data['pricetype'].str.lower() == 'retail'].copy()
            print(f"Filtered {len(df_retail):,} retail price records")
        else:
            df_retail = raw_data.copy()
            print(f"No price type column found. Using all {len(df_retail):,} records")

        # Initialize price_per_KG column
        if 'price' in df_retail.columns:
            df_retail['price_per_KG'] = df_retail['price']
            print("Using 'price' column as price_per_KG")
        else:
            df_retail['price_per_KG'] = 0
            print("No price column found, using 0 as default")

        # Drop rows with missing state values
        if 'state' in df_retail.columns:
            pre_count = len(df_retail)
            df_retail = df_retail[df_retail['state'].notna() & (df_retail['state'].str.strip() != '')]
            dropped = pre_count - len(df_retail)
            print(f"Dropped {dropped:,} rows with missing states ({dropped/pre_count:.2%})")
            print(f"Remaining records: {len(df_retail):,}")

        # Convert date to datetime
        self.date_col = self._find_date_column(df_retail)

        if self.date_col:
            print(f"Converting '{self.date_col}' to datetime")
            df_retail['date'] = pd.to_datetime(df_retail[self.date_col])
        else:
            print("❌ No date column found. Cannot proceed with time-based modeling")
            return None

        # Extract time features
        print("Extracting time features...")
        df_retail = self._add_time_features(df_retail)

        # Sort for logical imputation order
        df_retail = df_retail.sort_values(['state', 'date'])
        print("Sorted data by state and date")

        # Handle missing values
        print("Handling missing values...")
        df_retail = self._handle_missing_values(df_retail)

        # Filter for wheat only if commodity column exists
        if 'commodity' in df_retail.columns:
            pre_count = len(df_retail)
            wheat_data = df_retail[df_retail['commodity'].str.lower() == 'wheat'].copy()
            print(f"Filtered for wheat commodity: {len(wheat_data):,} records from {pre_count:,}")
        else:
            wheat_data = df_retail.copy()
            print(f"No commodity column found. Using all {len(wheat_data):,} records")

        print("✅ Preprocessing complete")

        # Display summary statistics
        print("\n📊 Summary Statistics for Preprocessed Data:")
        print(wheat_data.describe())

        return wheat_data

    def _find_date_column(self, df):
        """Find the date column in the dataframe."""
        date_candidates = ['date', 'date_x', 'Date']
        for col in date_candidates:
            if col in df.columns:
                return col
        return None

    def _add_time_features(self, df):
        """Add time-based features to the dataframe."""
        # Basic time components
        df['year'] = df['date'].dt.year
        df['month_num'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['quarter'] = df['date'].dt.quarter

        # Cyclical time features - these capture seasonality better
        df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)
        df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
        df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)

        print("Added time features: year, month, day, quarter")
        print("Added cyclical features: month_sin, month_cos, quarter_sin, quarter_cos")

        return df

    def _handle_missing_values(self, df):
        """Handle missing values in the dataframe."""
        # Fill Rainfall using state and month group mean
        if 'Rainfall' in df.columns:
            missing_before = df['Rainfall'].isna().sum()
            df['Rainfall'] = df.groupby(['state', 'month_num'])['Rainfall'].transform(
                lambda x: x.fillna(x.mean())
            )
            missing_after = df['Rainfall'].isna().sum()
            print(f"Filled {missing_before - missing_after:,} missing Rainfall values")

        # Fill diesel price forward within each state
        if 'Diesel Price' in df.columns:
            missing_before = df['Diesel Price'].isna().sum()
            df['diesel_price'] = df.groupby('state')['Diesel Price'].ffill()
            missing_after = df['diesel_price'].isna().sum()
            print(f"Filled {missing_before - missing_after:,} missing Diesel Price values")

        # Convert ROC columns to numeric and handle missing values
        if 'Diesel ROC' in df.columns:
            df['Diesel ROC'] = pd.to_numeric(df['Diesel ROC'], errors='coerce')
            missing_before = df['Diesel ROC'].isna().sum()
            df['Diesel ROC'].fillna(method='ffill', inplace=True)
            missing_after = df['Diesel ROC'].isna().sum()
            print(f"Filled {missing_before - missing_after:,} missing Diesel ROC values")

        if 'Wheat ROC' in df.columns:
            df['Wheat ROC'] = pd.to_numeric(df['Wheat ROC'], errors='coerce')
            missing_before = df['Wheat ROC'].isna().sum()
            df['Wheat ROC'].fillna(method='ffill', inplace=True)
            missing_after = df['Wheat ROC'].isna().sum()
            print(f"Filled {missing_before - missing_after:,} missing Wheat ROC values")

        # Calculate price ratios if not already present
        if 'diesel_price' in df.columns and 'price_per_KG' in df.columns:
            df['Diesel / Wheat Price Ratio'] = df['diesel_price'] / df['price_per_KG'].replace(0, np.nan)
            print("Calculated 'Diesel / Wheat Price Ratio'")

        # Report remaining missing values
        missing_counts = df.isnull().sum()
        if missing_counts.sum() > 0:
            print("\nRemaining missing values:")
            for col in missing_counts[missing_counts > 0].index:
                print(f"- {col}: {missing_counts[col]:,} missing values")
        else:
            print("No missing values remain in the dataset")

        return df


In [ ]:
class FeatureEngineer:
    """Class for feature engineering on time series data."""

    def engineer_features(self, df_state):
        """
        Add time series features to the data.

        Args:
            df_state (pandas.DataFrame): State data with date index

        Returns:
            pandas.DataFrame: Data with added features
        """
        print("Adding time series features...")

        # Set date as index if not already
        if not isinstance(df_state.index, pd.DatetimeIndex):
            if 'date' in df_state.columns:
                df_state = df_state.set_index('date', drop=False)
                df_state = df_state.sort_index()
                print("Set date as index and sorted data")

        # Add time lags for price
        for lag in [1, 3, 6, 12]:  # 1, 3, 6, 12 month lags
            lag_col = f'lag_{lag}m'
            df_state[lag_col] = df_state['price_per_KG'].shift(lag)
            print(f"Added {lag_col} feature")

        # Add rolling statistics
        for window in [3, 6]:
            # Rolling mean
            mean_col = f'rolling_mean_{window}m'
            df_state[mean_col] = df_state['price_per_KG'].rolling(window=window).mean()
            print(f"Added {mean_col} feature")

            # Rolling std
            std_col = f'rolling_std_{window}m'
            df_state[std_col] = df_state['price_per_KG'].rolling(window=window).std()
            print(f"Added {std_col} feature")

        # Calculate rate of change
        for period in [1, 3]:
            roc_col = f'roc_{period}m'
            df_state[roc_col] = df_state['price_per_KG'].pct_change(periods=period)
            print(f"Added {roc_col} feature")

        # Add MSP to retail price ratio
        if 'MSP_Wheat_KG' in df_state.columns:
            df_state['MSP_to_retail_ratio'] = df_state['MSP_Wheat_KG'] / df_state['price_per_KG']
            print("Added MSP_to_retail_ratio feature")

        # Report on engineered features
        feature_count = len(df_state.columns) - len(df_state.select_dtypes(include=['object']).columns)
        print(f"✅ Feature engineering complete. Dataset now has {feature_count} numeric features")

        # Display correlations with target
        print("\n📊 Correlation with price_per_KG:")
        # Select only numeric columns before calculating correlation
        numeric_df_state = df_state.select_dtypes(include=['number'])
        correlations = numeric_df_state.corr()['price_per_KG'].sort_values(ascending=False)
        print(correlations.head(10))

        return df_state


In [ ]:
class ModelTrainer:
    """Class for training and evaluating forecasting models with MLflow tracking."""

    def __init__(self, s3_connector, s3_bucket, output_prefix, mlflow_tracking_uri=None):
        """
        Initialize the model trainer with S3 storage and MLflow tracking.

        Args:
            s3_connector (S3Connector): S3 connector instance
            s3_bucket (str): S3 bucket for saving models and plots
            output_prefix (str): Prefix path in the S3 bucket
            mlflow_tracking_uri (str, optional): MLflow tracking URI
        """
        self.s3_connector = s3_connector
        self.s3_bucket = s3_bucket
        self.output_prefix = output_prefix
        self.feature_cols = None

        # Set up MLflow
        self.mlflow_uri = mlflow_tracking_uri
        if mlflow_tracking_uri:
            mlflow.set_tracking_uri(mlflow_tracking_uri)
            self.client = MlflowClient(tracking_uri=mlflow_tracking_uri)
            print(f"✅ MLflow tracking configured: {mlflow_tracking_uri}")
        else:
            self.client = None
            print("ℹ️ MLflow tracking not configured")

        # Create S3 directory structure
        self.model_registry_prefix = f"{output_prefix}/model_registry"
        self.plots_prefix = f"{output_prefix}/plots"

        # Ensure directories exist in S3
        self.s3_connector.create_s3_directory(s3_bucket, self.model_registry_prefix)
        self.s3_connector.create_s3_directory(s3_bucket, self.plots_prefix)

        print(f"✅ Set up S3 directory structure:")
        print(f"  - 📁 s3://{s3_bucket}/{self.model_registry_prefix} (models)")
        print(f"  - 📁 s3://{s3_bucket}/{self.plots_prefix} (visualizations)")

    def train_models_for_state(self, state, df_state, split_date):
        """
        Train models for a specific state.

        Args:
            state (str): State name
            df_state (pandas.DataFrame): Preprocessed data for the state
            split_date (str): Date to split train/test data

        Returns:
            dict: State model results
        """
        print(f"\n{'='*80}")
        print(f"📌 Processing state: {state.title()}")
        print(f"{'='*80}")

        try:
            # Check if we have enough data
            if len(df_state) < 24:  # Skip states with insufficient data
                print(f"⚠️ Skipping {state.title()} - insufficient data (only {len(df_state)} records)")
                return None

            print(f"Found {len(df_state):,} records for {state.title()}")

            # Define feature columns based on available columns
            self.feature_cols = self._get_feature_columns(df_state)

            # Prepare data for modeling
            print(f"Preparing train/test split using date {split_date}")

            # Split based on date
            train_data = df_state[df_state.index < split_date].copy()
            test_data = df_state[df_state.index >= split_date].copy()

            print(f"Train data: {len(train_data):,} records ({len(train_data)/len(df_state):.1%})")
            print(f"Test data: {len(test_data):,} records ({len(test_data)/len(df_state):.1%})")

            # Drop rows with NaN values in key columns
            train_data = train_data.dropna(subset=['price_per_KG'] +
                                         [col for col in self.feature_cols if col in train_data.columns])
            test_data = test_data.dropna(subset=['price_per_KG'] +
                                        [col for col in self.feature_cols if col in test_data.columns])

            print(f"After dropping NaN values - Train: {len(train_data):,}, Test: {len(test_data):,}")

            if len(train_data) < 12 or len(test_data) < 4:
                print(f"⚠️ Skipping {state.title()} - insufficient data after cleaning")
                return None

            # Feature scaling
            scaler_features = StandardScaler()

            # Extract features and target
            X_train = pd.DataFrame(
                scaler_features.fit_transform(train_data[self.feature_cols]),
                columns=self.feature_cols,
                index=train_data.index
            )
            y_train = train_data['price_per_KG'].values

            X_test = pd.DataFrame(
                scaler_features.transform(test_data[self.feature_cols]),
                columns=self.feature_cols,
                index=test_data.index
            )
            y_test = test_data['price_per_KG'].values

            print(f"Final feature matrix shapes - X_train: {X_train.shape}, X_test: {X_test.shape}")

            # Train models
            models = {}
            model_metrics = {}

            # Create MLflow experiment for the state if MLflow is configured
            if self.mlflow_uri:
                experiment_name = f"wheat_price_forecast_{state}"
                try:
                    experiment = self.client.get_experiment_by_name(experiment_name)
                    if experiment:
                        experiment_id = experiment.experiment_id
                    else:
                        experiment_id = mlflow.create_experiment(experiment_name)
                    print(f"✅ MLflow experiment: {experiment_name} (ID: {experiment_id})")
                except Exception as e:
                    print(f"⚠️ Error with MLflow experiment: {str(e)}")
                    experiment_id = None
            else:
                experiment_id = None

            # 1. Random Forest
            print("\n🔹 Training Random Forest model...")
            rf_model, rf_metrics, rf_importance = self._train_random_forest(
                X_train, y_train, X_test, y_test, state, experiment_id
            )
            models['random_forest'] = rf_model
            model_metrics['random_forest'] = rf_metrics

            # 2. XGBoost
            print("\n🔹 Training XGBoost model...")
            xgb_model, xgb_metrics, xgb_importance = self._train_xgboost(
                X_train, y_train, X_test, y_test, state, experiment_id
            )
            models['xgboost'] = xgb_model
            model_metrics['xgboost'] = xgb_metrics

            # 3. Holt-Winters
            print("\n🔹 Training Holt-Winters model...")
            hw_fit, hw_metrics = self._train_holt_winters(
                train_data, test_data, state, experiment_id
            )

            if hw_fit is not None:
                models['holt_winters'] = hw_fit
                model_metrics['holt_winters'] = hw_metrics

            # Determine best model
            print("\n🔹 Determining best model...")
            best_model_name = min(model_metrics, key=lambda x: model_metrics[x]['test_rmse'])
            best_model = models[best_model_name]
            best_metrics = model_metrics[best_model_name]

            print(f"Best model for {state.title()}: {best_model_name}")
            print(f"Best model metrics:")
            for metric, value in best_metrics.items():
                print(f"   - {metric}: {value:.4f}")

            # Log best model and comparison results to MLflow
            if self.mlflow_uri:
                with mlflow.start_run(experiment_id=experiment_id, run_name=f"best_model_comparison_{state}",nested=True):
                    mlflow.log_param("best_model", best_model_name)
                    for name, metrics in model_metrics.items():
                        for metric, value in metrics.items():
                            mlflow.log_metric(f"{name}_{metric}", value)

                    # Log a text summary
                    summary = f"Best model: {best_model_name}\n"
                    summary += f"Test RMSE: {best_metrics.get('test_rmse', 'N/A')}\n"
                    summary += f"Test R2: {best_metrics.get('test_r2', 'N/A')}\n"
                    mlflow.log_text(summary, "model_comparison_summary.txt")

            # Create visualizations and save directly to S3
            self._create_forecast_visualization(
                state,
                train_data,
                test_data,
                models,
                best_model_name,
                split_date
            )

            # Save all models directly to S3
            state_model_prefix = f"{self.model_registry_prefix}/{state}"
            self.s3_connector.create_s3_directory(self.s3_bucket, state_model_prefix)

            for model_name, model in models.items():
                s3_model_key = f"{state_model_prefix}/{model_name}.pkl"
                self.s3_connector.save_pickle_to_s3(model, self.s3_bucket, s3_model_key)
                print(f"Saved {model_name} model to s3://{self.s3_bucket}/{s3_model_key}")

            # Save best model separately
            best_model_key = f"{self.model_registry_prefix}/{state}_best_model.pkl"
            self.s3_connector.save_pickle_to_s3(best_model, self.s3_bucket, best_model_key)
            print(f"Saved best model ({best_model_name}) to s3://{self.s3_bucket}/{best_model_key}")

            # Store feature importance data
            if best_model_name == 'random_forest':
                importance_data = rf_importance
            elif best_model_name == 'xgboost':
                importance_data = xgb_importance
            else:
                importance_data = None

            # Create result structure with S3 paths instead of local paths
            result = {
                'best_model': best_model_name,
                'model': best_model,
                'metrics': best_metrics,
                'feature_importance': importance_data.to_dict() if importance_data is not None else None,
                'model_s3_path': f"s3://{self.s3_bucket}/{best_model_key}"
            }

            print(f"✅ Successfully processed {state.title()}")
            return result

        except Exception as e:
            print(f"❌ Error processing state {state}: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def _get_feature_columns(self, df):
        """Get feature columns that exist in the dataframe."""
        all_feature_cols = [
            # Economic indicators
            'MSP_Wheat_KG', 'CPI', 'diesel_price',
            'Diesel ROC', 'Wheat ROC', 'Diesel / Wheat Price Ratio',

            # External factors
            'Rainfall',

            # Time components
            'year', 'month_num', 'quarter',
            'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',

            # Lagged features
            'lag_1m', 'lag_3m', 'lag_6m', 'lag_12m',

            # Rolling statistics
            'rolling_mean_3m', 'rolling_mean_6m',
            'rolling_std_3m', 'rolling_std_6m',

            # Rate of change
            'roc_1m', 'roc_3m',

            # Price ratios
            'MSP_to_retail_ratio'
        ]

        # Filter to columns that exist in the dataframe
        feature_cols = [col for col in all_feature_cols if col in df.columns]
        print(f"Using {len(feature_cols)} features: {', '.join(feature_cols)}")

        return feature_cols

    def _train_random_forest(self, X_train, y_train, X_test, y_test, state, experiment_id=None):
        """Train and evaluate Random Forest model with MLflow tracking."""
        rf_params = {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'random_state': 42,
            'n_jobs': -1
        }

        print(f"RF Parameters: {rf_params}")

        # Start MLflow run if tracking is enabled
        if self.mlflow_uri and experiment_id:
            mlflow_run = mlflow.start_run(experiment_id=experiment_id, run_name=f"random_forest_{state}",nested=True)
            mlflow_run_id = mlflow_run.info.run_id
            print(f"Started MLflow run: {mlflow_run_id}")
        else:
            mlflow_run = None

        try:
            # Train model
            rf_model = RandomForestRegressor(**rf_params)
            rf_model.fit(X_train, y_train)

            # Evaluate
            train_pred_rf = rf_model.predict(X_train)
            test_pred_rf = rf_model.predict(X_test)

            rf_metrics = self._calculate_metrics(y_train, train_pred_rf, y_test, test_pred_rf)

            # Get feature importance
            feature_importance_rf = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': rf_model.feature_importances_
            }).sort_values('Importance', ascending=False)

            print(f"Random Forest metrics - Test RMSE: {rf_metrics['test_rmse']:.4f}, R²: {rf_metrics['test_r2']:.4f}")
            print(f"Top 5 important features:")
            for i, row in feature_importance_rf.head(5).iterrows():
                print(f"   - {row['Feature']}: {row['Importance']:.4f}")

            # Log to MLflow if tracking is enabled
            if mlflow_run:
                # Log parameters
                for param, value in rf_params.items():
                    mlflow.log_param(param, value)

                # Log metrics
                for metric, value in rf_metrics.items():
                    mlflow.log_metric(metric, value)

                # Log feature importance
                importance_table = feature_importance_rf.to_csv(index=False)
                mlflow.log_text(importance_table, "feature_importance.csv")

                # Create and log feature importance plot
                plt.figure(figsize=(10, 6))
                plt.bar(feature_importance_rf['Feature'].head(10), feature_importance_rf['Importance'].head(10))
                plt.xlabel('Features')
                plt.ylabel('Importance')
                plt.title('Random Forest Feature Importance')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "feature_importance.png")
                plt.close()

                # Log model
                mlflow.sklearn.log_model(rf_model, "random_forest_model")

                # Create and log residual plot
                plt.figure(figsize=(8, 6))
                plt.scatter(test_pred_rf, y_test - test_pred_rf, alpha=0.5)
                plt.axhline(y=0, color='r', linestyle='-')
                plt.xlabel('Predicted Values')
                plt.ylabel('Residuals')
                plt.title('Residual Plot')
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "residual_plot.png")
                plt.close()

                # Register model
                model_name = f"random_forest_{state}"
                mlflow.register_model(f"runs:/{mlflow_run_id}/random_forest_model", model_name)

                print(f"✅ Logged Random Forest model to MLflow: {model_name}")

        finally:
            # End the MLflow run
            if mlflow_run:
                mlflow.end_run()

        return rf_model, rf_metrics, feature_importance_rf

    def _train_xgboost(self, X_train, y_train, X_test, y_test, state, experiment_id=None):
        """Train and evaluate XGBoost model with MLflow tracking."""
        xgb_params = {
            'n_estimators': 1000,
            'learning_rate': 0.03,
            'max_depth': 6,
            'colsample_bytree': 0.8,
            'subsample': 0.9,
            'gamma': 0.1,
            'reg_alpha': 0.1,
            'reg_lambda': 0.5,
            'random_state': 42
        }

        print(f"XGBoost Parameters: {xgb_params}")

        # Start MLflow run if tracking is enabled
        if self.mlflow_uri and experiment_id:
            mlflow_run = mlflow.start_run(experiment_id=experiment_id, run_name=f"xgboost_{state}",nested=True)
            mlflow_run_id = mlflow_run.info.run_id
            print(f"Started MLflow run: {mlflow_run_id}")
        else:
            mlflow_run = None

        try:
            # Train model
            xgb_model = xgb.XGBRegressor(**xgb_params)
            xgb_model.fit(X_train, y_train)

            # Evaluate
            train_pred_xgb = xgb_model.predict(X_train)
            test_pred_xgb = xgb_model.predict(X_test)

            xgb_metrics = self._calculate_metrics(y_train, train_pred_xgb, y_test, test_pred_xgb)

            # Get feature importance
            feature_importance_xgb = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': xgb_model.feature_importances_
            }).sort_values('Importance', ascending=False)

            print(f"XGBoost metrics - Test RMSE: {xgb_metrics['test_rmse']:.4f}, R²: {xgb_metrics['test_r2']:.4f}")
            print(f"Top 5 important features:")
            for i, row in feature_importance_xgb.head(5).iterrows():
                print(f"   - {row['Feature']}: {row['Importance']:.4f}")

            # Log to MLflow if tracking is enabled
            if mlflow_run:
                # Log parameters
                for param, value in xgb_params.items():
                    mlflow.log_param(param, value)

                # Log metrics
                for metric, value in xgb_metrics.items():
                    mlflow.log_metric(metric, value)

                # Log feature importance
                importance_table = feature_importance_xgb.to_csv(index=False)
                mlflow.log_text(importance_table, "feature_importance.csv")

                # Create and log feature importance plot
                plt.figure(figsize=(10, 6))
                plt.bar(feature_importance_xgb['Feature'].head(10), feature_importance_xgb['Importance'].head(10))
                plt.xlabel('Features')
                plt.ylabel('Importance')
                plt.title('XGBoost Feature Importance')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "feature_importance.png")
                plt.close()

                # Log model
                mlflow.xgboost.log_model(xgb_model, "xgboost_model")

                # Create and log learning curves
                plt.figure(figsize=(8, 6))
                plt.scatter(test_pred_xgb, y_test - test_pred_xgb, alpha=0.5)
                plt.axhline(y=0, color='r', linestyle='-')
                plt.xlabel('Predicted Values')
                plt.ylabel('Residuals')
                plt.title('Residual Plot')
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "residual_plot.png")
                plt.close()

                # Register model
                model_name = f"xgboost_{state}"
                mlflow.register_model(f"runs:/{mlflow_run_id}/xgboost_model", model_name)

                print(f"✅ Logged XGBoost model to MLflow: {model_name}")

        finally:
            # End the MLflow run
            if mlflow_run:
                mlflow.end_run()

        return xgb_model, xgb_metrics, feature_importance_xgb

    def _train_holt_winters(self, train_data, test_data, state, experiment_id=None):
        """Train and evaluate Holt-Winters model with MLflow tracking."""
        # Convert to pandas Series with datetime index
        train_series = pd.Series(train_data['price_per_KG'].values, index=train_data.index)
        test_series = pd.Series(test_data['price_per_KG'].values, index=test_data.index)

        hw_params = {
            'trend': 'add',
            'seasonal': 'mul',
            'seasonal_periods': 12  # Monthly data with yearly seasonality
        }

        print(f"Holt-Winters Parameters: {hw_params}")

        # Start MLflow run if tracking is enabled
        if self.mlflow_uri and experiment_id:
            mlflow_run = mlflow.start_run(experiment_id=experiment_id, run_name=f"holt_winters_{state}",nested=True)
            mlflow_run_id = mlflow_run.info.run_id
            print(f"Started MLflow run: {mlflow_run_id}")
        else:
            mlflow_run = None

        try:
            # Create and fit model
            hw_model = ExponentialSmoothing(train_series, **hw_params)
            hw_fit = hw_model.fit()

            # Make predictions
            hw_train_pred = hw_fit.fittedvalues
            hw_test_pred = hw_fit.forecast(steps=len(test_series))

            # Align indices
            hw_test_pred.index = test_series.index

            # Calculate metrics
            hw_train_rmse = np.sqrt(mean_squared_error(train_series, hw_train_pred))
            hw_test_rmse = np.sqrt(mean_squared_error(test_series, hw_test_pred))
            hw_train_mae = mean_absolute_error(train_series, hw_train_pred)
            hw_test_mae = mean_absolute_error(test_series, hw_test_pred)

            # Avoid division by zero in MAPE
            hw_train_mape = np.mean(np.abs((train_series - hw_train_pred) /
                                          np.maximum(0.01, np.abs(train_series)))) * 100
            hw_test_mape = np.mean(np.abs((test_series - hw_test_pred) /
                                         np.maximum(0.01, np.abs(test_series)))) * 100

            # Calculate R² if possible
            try:
                hw_train_r2 = r2_score(train_series, hw_train_pred)
                hw_test_r2 = r2_score(test_series, hw_test_pred)
            except:
                hw_train_r2 = np.nan
                hw_test_r2 = np.nan

            hw_metrics = {
                'train_rmse': hw_train_rmse,
                'test_rmse': hw_test_rmse,
                'train_mae': hw_train_mae,
                'test_mae': hw_test_mae,
                'train_mape': hw_train_mape,
                'test_mape': hw_test_mape,
                'train_r2': hw_train_r2,
                'test_r2': hw_test_r2
            }

            print(f"Holt-Winters metrics - Test RMSE: {hw_test_rmse:.4f}")

            # Log to MLflow if tracking is enabled
            if mlflow_run:
                # Log parameters
                for param, value in hw_params.items():
                    mlflow.log_param(param, value)

                # Log model parameters from fitted model
                model_params = hw_fit.params
                for param, value in model_params.items():
                    param_name = f"fitted_{param}"
                    if isinstance(value, (int, float)):
                        mlflow.log_param(param_name, value)

                # Log metrics
                for metric, value in hw_metrics.items():
                    if not np.isnan(value):  # Skip NaN values
                        mlflow.log_metric(metric, value)

                # Create and log forecast visualization
                plt.figure(figsize=(12, 6))
                plt.plot(train_series.index, train_series, 'b-', label='Train Actual')
                plt.plot(hw_train_pred.index, hw_train_pred, 'g--', label='Train Fitted')
                plt.plot(test_series.index, test_series, 'r-', label='Test Actual')
                plt.plot(hw_test_pred.index, hw_test_pred, 'c--', label='Test Forecast')
                plt.xlabel('Date')
                plt.ylabel('Price per KG')
                plt.title(f'Holt-Winters Forecast - {state.title()}')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "forecast_plot.png")
                plt.close()

                # Create and log residual plot
                plt.figure(figsize=(10, 6))
                plt.scatter(hw_test_pred, test_series - hw_test_pred, alpha=0.5)
                plt.axhline(y=0, color='r', linestyle='-')
                plt.xlabel('Predicted Values')
                plt.ylabel('Residuals')
                plt.title('Residual Plot - Holt-Winters')
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "residual_plot.png")
                plt.close()

                # Save the model components
                model_info = {
                    'model_type': 'Holt-Winters',
                    'params': hw_params,
                    'fitted_params': {str(k): str(v) for k, v in model_params.items()},
                    'metrics': hw_metrics
                }
                mlflow.log_dict(model_info, "model_info.json")

                print(f"✅ Logged Holt-Winters model to MLflow")

            return hw_fit, hw_metrics

        except Exception as e:
            print(f"❌ Error training Holt-Winters model: {str(e)}")
            if mlflow_run:
                # Log the error
                mlflow.log_param("error", str(e))
            return None, None

        finally:
            # End the MLflow run
            if mlflow_run:
                mlflow.end_run()

    def _calculate_metrics(self, y_train, train_pred, y_test, test_pred):
        """Calculate common evaluation metrics."""
        train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
        train_mae = mean_absolute_error(y_train, train_pred)
        test_mae = mean_absolute_error(y_test, test_pred)
        train_r2 = r2_score(y_train, train_pred)
        test_r2 = r2_score(y_test, test_pred)

        # Avoid division by zero in MAPE
        train_mape = np.mean(np.abs((y_train - train_pred) / np.maximum(0.01, np.abs(y_train)))) * 100
        test_mape = np.mean(np.abs((y_test - test_pred) / np.maximum(0.01, np.abs(y_test)))) * 100

        metrics = {
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_mape': train_mape,
            'test_mape': test_mape,
            'train_r2': train_r2,
            'test_r2': test_r2
        }

        return metrics

    def _create_forecast_visualization(self, state, train_data, test_data, models, best_model_name, split_date):
        """Create visualization of model forecasts and save directly to S3."""
        print("\n🔹 Creating visualization...")

        plt.figure(figsize=(14, 8))

        # Plot actual prices
        plt.plot(train_data.index, train_data['price_per_KG'], 'b-',
                 label='Actual (Train)', alpha=0.7, linewidth=2)
        plt.plot(test_data.index, test_data['price_per_KG'], 'k-',
                 label='Actual (Test)', alpha=0.7, linewidth=2)

        # Plot best model predictions
        if best_model_name == 'holt_winters':
            hw_fit = models['holt_winters']
            hw_train_pred = hw_fit.fittedvalues
            hw_test_pred = hw_fit.forecast(steps=len(test_data))
            hw_test_pred.index = test_data.index

            plt.plot(hw_train_pred.index, hw_train_pred, 'g--',
                     label=f'Predicted (Train - {best_model_name})', alpha=0.7, linewidth=2)
            plt.plot(hw_test_pred.index, hw_test_pred, 'r--',
                     label=f'Predicted (Test - {best_model_name})', alpha=0.7, linewidth=2)
        else:
            # For RF and XGB, calculate predictions
            X_train = train_data[self.feature_cols]
            X_test = test_data[self.feature_cols]

            best_model = models[best_model_name]
            train_pred = best_model.predict(X_train)
            test_pred = best_model.predict(X_test)

            plt.plot(train_data.index, train_pred, 'g--',
                     label=f'Predicted (Train - {best_model_name})', alpha=0.7, linewidth=2)
            plt.plot(test_data.index, test_pred, 'r--',
                     label=f'Predicted (Test - {best_model_name})', alpha=0.7, linewidth=2)

        # Add chart elements
        plt.title(f'Wheat Price Forecast for {state.title()} - {best_model_name.title()}', fontsize=16)
        plt.xlabel('Date', fontsize=14)
        plt.ylabel('Price per KG (₹)', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='best', fontsize=12)

        # Add vertical line for train/test split
        plt.axvline(pd.to_datetime(split_date), color='gray', linestyle='--', alpha=0.7)
        plt.text(pd.to_datetime(split_date), plt.ylim()[0], 'Train-Test Split',
                 rotation=90, verticalalignment='bottom', fontsize=12)

        # Format plot
        plt.tight_layout()

        # Save the plot directly to S3
        s3_plot_key = f"{self.plots_prefix}/{state}_forecast.png"
        self.s3_connector.save_figure_to_s3(plt.gcf(), self.s3_bucket, s3_plot_key)
        plt.close()

        print(f"✅ Saved visualization to s3://{self.s3_bucket}/{s3_plot_key}")

        # Create feature importance plot if applicable
        if best_model_name in ['random_forest', 'xgboost']:
            self._create_feature_importance_plot(state, best_model_name, models[best_model_name], self.feature_cols)


    def _create_feature_importance_plot(self, state, model_name, model, feature_cols):
        """Create feature importance visualization and save directly to S3."""
        # Get feature importances
        importances = model.feature_importances_

        # Create DataFrame of feature importances
        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': importances
        }).sort_values('Importance', ascending=False)

        # Create plot
        plt.figure(figsize=(12, 8))

        # Plot horizontal bar chart
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15), palette='viridis')

        # Add chart elements
        plt.title(f'Top 15 Feature Importances for {state.title()} - {model_name.title()}', fontsize=16)
        plt.xlabel('Importance', fontsize=14)
        plt.ylabel('Feature', fontsize=14)
        plt.grid(True, alpha=0.3, axis='x')

        # Format plot
        plt.tight_layout()

        # Save the plot directly to S3
        s3_plot_key = f"{self.plots_prefix}/{state}_{model_name}_importance.png"
        self.s3_connector.save_figure_to_s3(plt.gcf(), self.s3_bucket, s3_plot_key)
        plt.close()

        print(f"✅ Saved feature importance plot to s3://{self.s3_bucket}/{s3_plot_key}")

        # Also save the feature importance data as JSON
        importance_data = feature_importance.to_dict('records')
        s3_importance_key = f"{self.plots_prefix}/{state}_{model_name}_importance.json"
        self.s3_connector.save_data_to_s3(
            importance_data,
            self.s3_bucket,
            s3_importance_key,
            content_type='application/json'
        )
        print(f"✅ Saved feature importance data to s3://{self.s3_bucket}/{s3_importance_key}")

In [ ]:
class ModelAggregator:
    """Class for aggregating models across all states with MLflow tracking."""

    def __init__(self, s3_connector, s3_bucket, output_prefix, mlflow_tracking_uri=None):
        """
        Initialize the model aggregator with S3 storage and MLflow tracking.

        Args:
            s3_connector (S3Connector): S3 connector instance
            s3_bucket (str): S3 bucket for saving models
            output_prefix (str): Prefix path in the S3 bucket
            mlflow_tracking_uri (str, optional): MLflow tracking URI
        """
        self.s3_connector = s3_connector
        self.s3_bucket = s3_bucket
        self.output_prefix = output_prefix

        # Set up MLflow
        self.mlflow_uri = mlflow_tracking_uri
        if mlflow_tracking_uri:
            mlflow.set_tracking_uri(mlflow_tracking_uri)
            self.client = MlflowClient(tracking_uri=mlflow_tracking_uri)
            print(f"✅ MLflow tracking configured for aggregator: {mlflow_tracking_uri}")
        else:
            self.client = None
            print("ℹ️ MLflow tracking not configured for aggregator")

    def aggregate_models(self, all_states_models, feature_cols):
        """
        Combine all state models into a single file with metadata and save to S3.

        Args:
            all_states_models (dict): Dictionary of state-model mappings
            feature_cols (list): Feature columns used in modeling

        Returns:
            str: S3 path to the combined model file
        """
        print("\n" + "="*80)
        print("📦 Aggregating models across all states to S3")
        print("="*80)

        # Filter out None values
        valid_models = {state: data for state, data in all_states_models.items() if data is not None}

        print(f"Aggregating models for {len(valid_models)}/{len(all_states_models)} states")

        # Create metadata
        metadata = {
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'states_included': list(valid_models.keys()),
            'total_states': len(valid_models),
            'model_details': {
                state: {
                    'best_model': data['best_model'],
                    'metrics': data['metrics'],
                    'feature_importance': data['feature_importance'],
                    's3_model_path': data.get('model_s3_path', f"s3://{self.s3_bucket}/{self.output_prefix}/model_registry/{state}_best_model.pkl")
                } for state, data in valid_models.items()
            },
            'feature_cols': feature_cols
        }

        # Create combined data structure
        combined_data = {
            'models': {state: data['model'] for state, data in valid_models.items()},
            'metadata': metadata
        }

        # Generate S3 paths with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Save the combined model directly to S3
        s3_combined_key = f"{self.output_prefix}/models/all_states_best_model_{timestamp}.pkl"
        s3_latest_key = f"{self.output_prefix}/models/all_states_best_model_latest.pkl"
        s3_metadata_key = f"{self.output_prefix}/models/all_states_metadata_{timestamp}.json"

        # Start MLflow run if tracking is enabled
        if self.mlflow_uri:
            # Create a new experiment for aggregated models if it doesn't exist
            experiment_name = "wheat_price_forecast_aggregated"
            try:
                experiment = self.client.get_experiment_by_name(experiment_name)
                if experiment:
                    experiment_id = experiment.experiment_id
                else:
                    experiment_id = mlflow.create_experiment(experiment_name)
                print(f"✅ MLflow experiment for aggregation: {experiment_name} (ID: {experiment_id})")

                # Start a new run
                mlflow_run = mlflow.start_run(experiment_id=experiment_id, run_name=f"aggregated_models_{timestamp}",nested=True)
                run_id = mlflow_run.info.run_id
                print(f"Started MLflow run: {run_id}")

                # Log metadata about the aggregation
                mlflow.log_param("timestamp", timestamp)
                mlflow.log_param("total_states", len(valid_models))
                mlflow.log_param("states", ",".join(sorted(valid_models.keys())))

                # Log performance metrics for each state
                for state, data in valid_models.items():
                    best_model = data['best_model']
                    metrics = data['metrics']

                    # Log the best model type for each state
                    mlflow.log_param(f"{state}_best_model", best_model)

                    # Log key metrics for each state
                    for metric_name in ['test_rmse', 'test_r2', 'test_mape']:
                        if metric_name in metrics:
                            mlflow.log_metric(f"{state}_{metric_name}", metrics[metric_name])

                # Create and log a summary table of results
                results_table = self._create_results_table(valid_models)
                mlflow.log_text(results_table, "state_model_results.csv")

                # Log feature importance across all states
                combined_importance = self._aggregate_feature_importance(valid_models, feature_cols)
                if combined_importance is not None:
                    mlflow.log_dict(combined_importance, "combined_feature_importance.json")

                    # Create and log feature importance plot
                    plt.figure(figsize=(10, 8))
                    top_features = sorted(combined_importance.items(), key=lambda x: x[1], reverse=True)[:15]
                    features, importances = zip(*top_features)
                    plt.barh(features, importances)
                    plt.xlabel('Aggregated Importance')
                    plt.ylabel('Features')
                    plt.title('Top 15 Features Across All States')
                    plt.tight_layout()
                    mlflow.log_figure(plt.gcf(), "aggregated_feature_importance.png")
                    plt.close()

                # Log the S3 paths
                mlflow.log_param("s3_combined_model_path", f"s3://{self.s3_bucket}/{s3_combined_key}")
                mlflow.log_param("s3_latest_model_path", f"s3://{self.s3_bucket}/{s3_latest_key}")
                mlflow.log_param("s3_metadata_path", f"s3://{self.s3_bucket}/{s3_metadata_key}")

                # Register the aggregated model metadata
                mlflow.log_dict(metadata, "aggregated_model_metadata.json")

                print(f"✅ Logged aggregated model metadata to MLflow")

            except Exception as e:
                print(f"⚠️ Error with MLflow experiment: {str(e)}")

        # Save combined model to S3
        model_saved = self.s3_connector.save_pickle_to_s3(
            combined_data,
            self.s3_bucket,
            s3_combined_key
        )

        if model_saved:
            print(f"✅ Saved combined model to S3: s3://{self.s3_bucket}/{s3_combined_key}")

            # Save a copy as "latest" version
            self.s3_connector.save_pickle_to_s3(
                combined_data,
                self.s3_bucket,
                s3_latest_key
            )
            print(f"✅ Saved latest version to S3: s3://{self.s3_bucket}/{s3_latest_key}")

            # Save metadata as JSON
            self.s3_connector.save_data_to_s3(
                metadata,
                self.s3_bucket,
                s3_metadata_key,
                content_type='application/json'
            )
            print(f"✅ Saved metadata to S3: s3://{self.s3_bucket}/{s3_metadata_key}")
        else:
            print(f"❌ Failed to save combined model to S3")

        # End the MLflow run if it was started
        if self.mlflow_uri:
            try:
                mlflow.end_run()
            except Exception as e:
                print(f"⚠️ Error ending MLflow run: {str(e)}")

        # Return the S3 path
        combined_s3_path = f"s3://{self.s3_bucket}/{s3_combined_key}"
        return combined_s3_path

    def _create_results_table(self, valid_models):
        """Create a CSV-formatted table of results for all states."""
        # Header row
        table = "State,Best Model,Test RMSE,Test MAPE,R²\n"

        # Add a row for each state
        for state, details in sorted(valid_models.items()):
            rmse = details['metrics'].get('test_rmse', 'N/A')
            mape = details['metrics'].get('test_mape', 'N/A')
            r2 = details['metrics'].get('test_r2', 'N/A')

            rmse_str = f"{rmse:.4f}" if isinstance(rmse, (int, float)) else rmse
            mape_str = f"{mape:.2f}" if isinstance(mape, (int, float)) else mape
            r2_str = f"{r2:.4f}" if isinstance(r2, (int, float)) else r2

            table += f"{state},{details['best_model']},{rmse_str},{mape_str},{r2_str}\n"

        return table

    def _aggregate_feature_importance(self, valid_models, feature_cols):
        """Aggregate feature importance across all models."""
        # Initialize dictionary to hold importance scores
        combined_importance = {feature: 0.0 for feature in feature_cols}
        models_with_importance = 0

        # Sum importance scores across all states
        for state, data in valid_models.items():
            # Skip if no feature importance data
            if data['feature_importance'] is None:
                continue

            models_with_importance += 1

            # Extract feature importance dict from the data
            importance_dict = data['feature_importance']

            # If it's a DataFrame in dict form, extract the values
            if isinstance(importance_dict, dict) and 'Feature' in importance_dict and 'Importance' in importance_dict:
                features = importance_dict['Feature']
                importances = importance_dict['Importance']

                # Combine the feature and importance values
                for i, feature in enumerate(features):
                    if feature in combined_importance:
                        combined_importance[feature] += importances[i]

        # Average the importance scores
        if models_with_importance > 0:
            for feature in combined_importance:
                combined_importance[feature] /= models_with_importance
            return combined_importance
        else:
            return None

    def print_summary_table(self, all_states_models):
        """
        Print a summary table of model results.

        Args:
            all_states_models (dict): Dictionary of state-model mappings
        """
        print("\n" + "="*80)
        print("📊 Summary of Results")
        print("="*80)

        # Filter out None values
        valid_models = {state: data for state, data in all_states_models.items() if data is not None}

        # Print header
        print(f"{'State':<20} {'Best Model':<15} {'Test RMSE':<12} {'Test MAPE':<12} {'R²':<8}")
        print(f"{'-'*80}")

        # Print each state's results
        for state, details in sorted(valid_models.items()):
            rmse = details['metrics'].get('test_rmse', 'N/A')
            mape = details['metrics'].get('test_mape', 'N/A')
            r2 = details['metrics'].get('test_r2', 'N/A')

            rmse_str = f"{rmse:.4f}" if isinstance(rmse, (int, float)) else rmse
            mape_str = f"{mape:.2f}%" if isinstance(mape, (int, float)) else mape
            r2_str = f"{r2:.4f}" if isinstance(r2, (int, float)) else r2

            print(f"{state.title():<20} {details['best_model']:<15} {rmse_str:<12} {mape_str:<12} {r2_str:<8}")

        print(f"{'='*80}")
        print(f"✅ Successfully processed {len(valid_models)}/{len(all_states_models)} states")

        # Log to MLflow if tracking is enabled
        if self.mlflow_uri:
            try:
                # Create a separate experiment for the summary
                experiment_name = "wheat_price_forecast_summary"
                experiment = self.client.get_experiment_by_name(experiment_name)
                if experiment:
                    experiment_id = experiment.experiment_id
                else:
                    experiment_id = mlflow.create_experiment(experiment_name)

                # Start a new run
                with mlflow.start_run(experiment_id=experiment_id, run_name=f"model_performance_summary",nested=True):
                    # Log summary metrics
                    test_rmse_values = [data['metrics'].get('test_rmse') for data in valid_models.values()
                                       if isinstance(data['metrics'].get('test_rmse'), (int, float))]

                    if test_rmse_values:
                        mlflow.log_metric("mean_test_rmse", np.mean(test_rmse_values))
                        mlflow.log_metric("median_test_rmse", np.median(test_rmse_values))
                        mlflow.log_metric("min_test_rmse", np.min(test_rmse_values))
                        mlflow.log_metric("max_test_rmse", np.max(test_rmse_values))

                    # Log model type distribution
                    model_types = [data['best_model'] for data in valid_models.values()]
                    model_counts = {}
                    for model_type in set(model_types):
                        count = model_types.count(model_type)
                        model_counts[model_type] = count
                        mlflow.log_metric(f"{model_type}_count", count)

                    # Log total states processed
                    mlflow.log_param("total_states_processed", len(valid_models))
                    mlflow.log_param("total_states_attempted", len(all_states_models))

                    # Create and log a summary visualization
                    self._create_summary_visualizations(valid_models)

                print(f"✅ Logged summary statistics to MLflow")

            except Exception as e:
                print(f"⚠️ Error logging summary to MLflow: {str(e)}")

    def _create_summary_visualizations(self, valid_models):
        """Create and log summary visualizations to MLflow."""
        try:
            # 1. Model distribution pie chart
            model_types = [data['best_model'] for data in valid_models.values()]
            model_counts = {}
            for model_type in set(model_types):
                model_counts[model_type] = model_types.count(model_type)

            plt.figure(figsize=(8, 8))
            plt.pie(model_counts.values(), labels=model_counts.keys(), autopct='%1.1f%%',
                   shadow=True, startangle=90)
            plt.title('Distribution of Best Model Types')
            plt.axis('equal')
            mlflow.log_figure(plt.gcf(), "model_distribution_pie.png")
            plt.close()

            # 2. RMSE distribution histogram
            rmse_values = [data['metrics'].get('test_rmse') for data in valid_models.values()
                          if isinstance(data['metrics'].get('test_rmse'), (int, float))]

            if rmse_values:
                plt.figure(figsize=(10, 6))
                plt.hist(rmse_values, bins=10, alpha=0.7, color='skyblue')
                plt.axvline(np.mean(rmse_values), color='red', linestyle='dashed', linewidth=1,
                           label=f'Mean RMSE: {np.mean(rmse_values):.4f}')
                plt.axvline(np.median(rmse_values), color='green', linestyle='dashed', linewidth=1,
                           label=f'Median RMSE: {np.median(rmse_values):.4f}')
                plt.xlabel('Test RMSE')
                plt.ylabel('Number of States')
                plt.title('Distribution of Test RMSE Across States')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "rmse_distribution.png")
                plt.close()

            # 3. Top performers bar chart
            top_states = sorted([(state, data['metrics'].get('test_rmse', float('inf')))
                                for state, data in valid_models.items()
                                if isinstance(data['metrics'].get('test_rmse'), (int, float))],
                               key=lambda x: x[1])[:5]

            if top_states:
                states, rmse_values = zip(*top_states)
                plt.figure(figsize=(10, 6))
                plt.bar(states, rmse_values, color='lightgreen')
                plt.xlabel('State')
                plt.ylabel('Test RMSE (lower is better)')
                plt.title('Top 5 Performing States')
                plt.xticks(rotation=45)
                plt.grid(True, alpha=0.3, axis='y')
                plt.tight_layout()
                mlflow.log_figure(plt.gcf(), "top_performers.png")
                plt.close()

        except Exception as e:
            print(f"⚠️ Error creating summary visualizations: {str(e)}")

In [ ]:
class WheatPriceForecaster:
    """Main class for wheat price forecasting process with S3 storage and MLflow tracking."""

    def __init__(self,
                 s3_bucket,
                 s3_key,
                 output_prefix="wheat_forecaster",
                 split_date="2020-01-01",
                 mlflow_uri=None,
                 aws_access_key_id=None,
                 aws_secret_access_key=None,
                 aws_region='us-east-1'):
        """
        Initialize the wheat price forecasting process with S3 data source and storage.

        Args:
            s3_bucket (str): S3 bucket containing the CSV data file and for storing outputs
            s3_key (str): S3 key (path) to the CSV data file
            output_prefix (str): Prefix path in the S3 bucket for all outputs
            split_date (str): Date to split train/test data
            mlflow_uri (str, optional): MLflow tracking URI
            aws_access_key_id (str, optional): AWS access key ID
            aws_secret_access_key (str, optional): AWS secret access key
            aws_region (str, optional): AWS region name
        """
        self.s3_bucket = s3_bucket
        self.s3_key = s3_key
        self.output_prefix = output_prefix
        self.split_date = split_date
        self.mlflow_uri = mlflow_uri

        # Initialize S3 connector
        self.s3_connector = S3Connector(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            region_name=aws_region
        )

        # Initialize other attributes
        self.raw_data = None
        self.wheat_data = None
        self.all_states = None
        self.all_state_models = {}
        self.feature_cols = None

        # Print initialization information
        print(f"🚀 Initializing Wheat Price Forecaster with S3 storage")
        print(f"📄 Input file: s3://{self.s3_bucket}/{self.s3_key}")
        print(f"📁 Output location: s3://{self.s3_bucket}/{self.output_prefix}/")
        print(f"📅 Train/Test split date: {self.split_date}")

        # Set up MLflow if provided
        if self.mlflow_uri:
            print(f"📊 MLflow tracking URI: {self.mlflow_uri}")
            mlflow.set_tracking_uri(self.mlflow_uri)
            self.client = MlflowClient(tracking_uri=self.mlflow_uri)

            # Create the main experiment
            try:
                experiment_name = "wheat_price_forecaster"
                experiment = self.client.get_experiment_by_name(experiment_name)
                if experiment:
                    self.main_experiment_id = experiment.experiment_id
                    print(f"Using existing MLflow experiment: {experiment_name} (ID: {self.main_experiment_id})")
                else:
                    self.main_experiment_id = mlflow.create_experiment(experiment_name)
                    print(f"Created new MLflow experiment: {experiment_name} (ID: {self.main_experiment_id})")
            except Exception as e:
                print(f"⚠️ Error setting up MLflow experiment: {str(e)}")
                self.main_experiment_id = None
        else:
            self.client = None
            self.main_experiment_id = None

    def run_full_process(self):
        """Run the complete wheat price forecasting process with S3 storage and MLflow tracking."""
        # Start a main run in MLflow if configured
        if self.mlflow_uri and self.main_experiment_id:
            main_run = mlflow.start_run(experiment_id=self.main_experiment_id, run_name="full_forecasting_process",nested=True)
            main_run_id = main_run.info.run_id
            print(f"Started main MLflow run: {main_run_id}")

            # Log key parameters
            mlflow.log_param("s3_bucket", self.s3_bucket)
            mlflow.log_param("s3_key", self.s3_key)
            mlflow.log_param("output_prefix", self.output_prefix)
            mlflow.log_param("split_date", self.split_date)
            mlflow.log_param("start_time", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        else:
            main_run = None

        try:
            # Step 1: Create S3 directory structure
            print("\n" + "="*80)
            print("Step 1: Setting up S3 directory structure")
            print("="*80)

            # Create S3 directory structure
            self.s3_connector.create_s3_directory(self.s3_bucket, f"{self.output_prefix}/")
            self.s3_connector.create_s3_directory(self.s3_bucket, f"{self.output_prefix}/model_registry/")
            self.s3_connector.create_s3_directory(self.s3_bucket, f"{self.output_prefix}/plots/")
            self.s3_connector.create_s3_directory(self.s3_bucket, f"{self.output_prefix}/models/")

            print(f"✅ Created S3 directory structure in s3://{self.s3_bucket}/{self.output_prefix}/")

            # Step 2: Load data from S3
            print("\n" + "="*80)
            print("Step 2: Loading data from S3")
            print("="*80)
            self.raw_data = self.s3_connector.read_csv_from_s3(self.s3_bucket, self.s3_key)
            if self.raw_data is None:
                print("❌ Failed to load data from S3. Exiting process.")
                if main_run:
                    mlflow.log_param("status", "FAILED")
                    mlflow.log_param("error", "Failed to load data from S3")
                    mlflow.end_run()
                return None

            # Log data statistics to MLflow
            if main_run:
                mlflow.log_param("data_rows", len(self.raw_data))
                mlflow.log_param("data_columns", len(self.raw_data.columns))

                # Log data schema
                schema = self.raw_data.dtypes.astype(str).to_dict()
                mlflow.log_dict(schema, "data_schema.json")

            # Step 3: Preprocess data
            print("\n" + "="*80)
            print("Step 3: Preprocessing data")
            print("="*80)
            preprocessor = DataPreprocessor()
            self.wheat_data = preprocessor.preprocess_retail_prices(self.raw_data)
            if self.wheat_data is None:
                print("❌ Preprocessing failed. Exiting process.")
                if main_run:
                    mlflow.log_param("status", "FAILED")
                    mlflow.log_param("error", "Preprocessing failed")
                    mlflow.end_run()
                return None

            # Log preprocessed data stats to MLflow
            if main_run:
                mlflow.log_param("preprocessed_rows", len(self.wheat_data))

                # Log summary statistics
                summary_stats = self.wheat_data.describe().to_dict()
                mlflow.log_dict(summary_stats, "summary_statistics.json")

            # Get all states
            self.all_states = self.wheat_data['state'].str.lower().unique().tolist()
            print(f"\nFound {len(self.all_states)} states with wheat price data:")
            for i, state in enumerate(self.all_states):
                print(f"   {i+1}. {state.title()}")

            if main_run:
                mlflow.log_param("total_states", len(self.all_states))
                mlflow.log_param("states", ",".join(sorted(self.all_states)))

            # Step 4: Process each state
            print("\n" + "="*80)
            print("Step 4: Processing individual states")
            print("="*80)

            feature_engineer = FeatureEngineer()
            model_trainer = ModelTrainer(
                s3_connector=self.s3_connector,
                s3_bucket=self.s3_bucket,
                output_prefix=self.output_prefix,
                mlflow_tracking_uri=self.mlflow_uri
            )

            for state_idx, state in enumerate(self.all_states):
                print(f"\nProcessing state {state_idx+1}/{len(self.all_states)}: {state.title()}")

                # Filter data for this state
                df_state = self.wheat_data[self.wheat_data['state'].str.lower() == state.lower()].copy()

                # Engineer features
                df_state = feature_engineer.engineer_features(df_state)

                # Train models for this state
                state_result = model_trainer.train_models_for_state(state, df_state, self.split_date)

                if state_result is not None:
                    # Store feature columns from first successful state if not already set
                    if self.feature_cols is None:
                        self.feature_cols = model_trainer.feature_cols

                    # Store state result
                    self.all_state_models[state] = state_result

                    # Log information about this state to the main MLflow run
                    if main_run:
                        mlflow.log_metric(f"{state}_test_rmse", state_result['metrics']['test_rmse'])
                        if 'test_r2' in state_result['metrics']:
                            mlflow.log_metric(f"{state}_test_r2", state_result['metrics']['test_r2'])

            # Step 5: Aggregate models to S3
            print("\n" + "="*80)
            print("Step 5: Aggregating models to S3")
            print("="*80)

            # Initialize aggregator with S3 capability and MLflow tracking
            aggregator = ModelAggregator(
                s3_connector=self.s3_connector,
                s3_bucket=self.s3_bucket,
                output_prefix=self.output_prefix,
                mlflow_tracking_uri=self.mlflow_uri
            )
            combined_s3_path = aggregator.aggregate_models(self.all_state_models, self.feature_cols)

            # Step 6: Print summary
            print("\n" + "="*80)
            print("Step 6: Results summary")
            print("="*80)

            aggregator.print_summary_table(self.all_state_models)

            # Log successful states count to MLflow
            if main_run:
                valid_models_count = len([model for model in self.all_state_models.values() if model is not None])
                mlflow.log_metric("successful_states", valid_models_count)
                mlflow.log_metric("success_rate", valid_models_count / len(self.all_states))
                mlflow.log_param("combined_model_path", combined_s3_path)
                mlflow.log_param("status", "SUCCESS")
                mlflow.log_param("end_time", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

            print("\n" + "="*80)
            print("✅ Wheat price forecasting process completed successfully")
            print(f"✅ All outputs saved to S3: s3://{self.s3_bucket}/{self.output_prefix}/")
            print("="*80)

            return {
                'combined_model_s3_path': combined_s3_path,
                'all_state_models': self.all_state_models,
                'feature_cols': self.feature_cols,
                's3_location': f"s3://{self.s3_bucket}/{self.output_prefix}/"
            }

        except Exception as e:
            print(f"❌ Error in forecasting process: {str(e)}")
            import traceback
            traceback.print_exc()

            # Log the error to MLflow
            if main_run:
                mlflow.log_param("status", "ERROR")
                mlflow.log_param("error_message", str(e))
                mlflow.log_text(traceback.format_exc(), "error_traceback.txt")

            return None

        finally:
            # End the main MLflow run if it was started
            if main_run:
                mlflow.end_run()

In [ ]:
def main(s3_bucket,
         s3_key,
         output_prefix="wheat_forecaster",
         split_date="2020-01-01",
         mlflow_uri=None,
         aws_access_key_id=None,
         aws_secret_access_key=None,
         aws_region='us-east-1'):
    """
    Run the wheat price forecasting pipeline using data from S3 and storing all outputs to S3.

    Args:
        s3_bucket (str): S3 bucket containing the CSV data file and for storing outputs
        s3_key (str): S3 key (path) to the CSV data file
        output_prefix (str): Prefix path in the S3 bucket for all outputs
        split_date (str): Date to split train/test data
        mlflow_uri (str, optional): MLflow tracking URI
        aws_access_key_id (str, optional): AWS access key ID
        aws_secret_access_key (str, optional): AWS secret access key
        aws_region (str, optional): AWS region name

    Returns:
        dict: Results from the forecasting process with S3 paths
    """
    # Configure MLflow if provided
    if aws_access_key_id and aws_secret_access_key:
        os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
        os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
        os.environ["AWS_DEFAULT_REGION"] = aws_region
    if mlflow_uri:
        # Set tracking URI
        mlflow.set_tracking_uri(mlflow_uri)
        print(f"MLflow tracking URI set to: {mlflow_uri}")

        # Create main experiment if it doesn't exist
        experiment_name = "wheat_price_forecaster_main"
        try:
            experiment = mlflow.get_experiment_by_name(experiment_name)
            if experiment:
                experiment_id = experiment.experiment_id
                print(f"Using existing MLflow experiment: {experiment_name} (ID: {experiment_id})")
            else:
                experiment_id = mlflow.create_experiment(experiment_name)
                print(f"Created new MLflow experiment: {experiment_name} (ID: {experiment_id})")

            # Start the main run
            with mlflow.start_run(experiment_id=experiment_id, run_name="pipeline_execution",nested=True):
                mlflow.log_param("execution_timestamp", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                mlflow.log_param("s3_bucket", s3_bucket)
                mlflow.log_param("s3_key", s3_key)
                mlflow.log_param("output_prefix", output_prefix)
                mlflow.log_param("split_date", split_date)
                mlflow.log_param("aws_region", aws_region)

                # Initialize and run the forecaster
                forecaster = WheatPriceForecaster(
                    s3_bucket=s3_bucket,
                    s3_key=s3_key,
                    output_prefix=output_prefix,
                    split_date=split_date,
                    mlflow_uri=mlflow_uri,
                    aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    aws_region=aws_region
                )
                results = forecaster.run_full_process()
                if results:
                    # Add wheat_data to the results dictionary
                    results['wheat_data'] = forecaster.wheat_data

                    # # Now run the evaluation
                    # evaluation_results = evaluate_wheat_price_models(results)
                # Log final results
                if results:
                    mlflow.log_param("status", "SUCCESS")
                    mlflow.log_param("combined_model_path", results.get('combined_model_s3_path', 'None'))
                    mlflow.log_param("feature_count", len(results.get('feature_cols', [])))
                    mlflow.log_param("states_processed", len(results.get('all_state_models', {})))
                else:
                    mlflow.log_param("status", "FAILED")

                return results

        except Exception as e:
            print(f"Error with MLflow setup: {str(e)}")
            # Fall back to running without MLflow tracking for this specific run
            forecaster = WheatPriceForecaster(
                s3_bucket=s3_bucket,
                s3_key=s3_key,
                output_prefix=output_prefix,
                split_date=split_date,
                mlflow_uri=None,  # Don't try to use MLflow
                aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                aws_region=aws_region
            )
            results = forecaster.run_full_process()
            return results
    else:
        # Run without MLflow
        forecaster = WheatPriceForecaster(
            s3_bucket=s3_bucket,
            s3_key=s3_key,
            output_prefix=output_prefix,
            split_date=split_date,
            mlflow_uri=None,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_region=aws_region
        )
        results = forecaster.run_full_process()
        return results

In [ ]:
import os
# os.environ['AWS_ACCESS_KEY_ID'] = ''
# os.environ['AWS_SECRET_ACCESS_KEY'] = ''
# os.environ['AWS_DEFAULT_REGION'] = ''

In [ ]:
results = main(
    s3_bucket="foundation-project-data",
    s3_key="data/wheat_prices_merged.csv",
    output_prefix="wheat_forecaster",
    split_date="2020-01-01",
    mlflow_uri="http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/",
    #aws_access_key_id="",
    #aws_secret_access_key="",
    #aws_region=""
)

MLflow tracking URI set to: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/
Using existing MLflow experiment: wheat_price_forecaster_main (ID: 655825255592778811)
✅ S3 client initialized for region: eu-north-1
🚀 Initializing Wheat Price Forecaster with S3 storage
📄 Input file: s3://foundation-project-data/data/wheat_prices_merged.csv
📁 Output location: s3://foundation-project-data/wheat_forecaster/
📅 Train/Test split date: 2020-01-01
📊 MLflow tracking URI: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/
Using existing MLflow experiment: wheat_price_forecaster (ID: 957387433427483073)
Started main MLflow run: 4f69b5e6c00d4f55807d8af16a33169e

Step 1: Setting up S3 directory structure
✅ Created S3 directory: s3://foundation-project-data/wheat_forecaster/
✅ Created S3 directory: s3://foundation-project-data/wheat_forecaster/model_registry/
✅ Created S3 directory: s3://foundation-project-data/wheat_forecaster/plots/
✅ Created S3 directory: s3://foundation-pr

2025/04/20 07:04:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'random_forest_andaman and nicobar' already exists. Creating a new version of this model...
2025/04/20 07:04:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_andaman and nicobar, version 4
Created version '4' of model 'random_forest_andaman and nicobar'.


✅ Logged Random Forest model to MLflow: random_forest_andaman and nicobar
🏃 View run random_forest_andaman and nicobar at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254/runs/9ead936e10f346bda3194b4b7ebbe9bf
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254

🔹 Training XGBoost model...
XGBoost Parameters: {'n_estimators': 1000, 'learning_rate': 0.03, 'max_depth': 6, 'colsample_bytree': 0.8, 'subsample': 0.9, 'gamma': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.5, 'random_state': 42}
Started MLflow run: c2597dbf0fbf4981978e5f2c8fb43b3f
XGBoost metrics - Test RMSE: 3.3279, R²: -5.0854
Top 5 important features:
   - rolling_mean_6m: 0.4372
   - rolling_mean_3m: 0.1423
   - MSP_Wheat_KG: 0.1131
   - year: 0.1039
   - diesel_price: 0.0922


2025/04/20 07:04:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgboost_andaman and nicobar' already exists. Creating a new version of this model...
2025/04/20 07:04:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost_andaman and nicobar, version 3
Created version '3' of model 'xgboost_andaman and nicobar'.


✅ Logged XGBoost model to MLflow: xgboost_andaman and nicobar
🏃 View run xgboost_andaman and nicobar at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254/runs/c2597dbf0fbf4981978e5f2c8fb43b3f
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254

🔹 Training Holt-Winters model...
Holt-Winters Parameters: {'trend': 'add', 'seasonal': 'mul', 'seasonal_periods': 12}
Started MLflow run: cc59117c0e08459ba1c1b328eb09081c
Holt-Winters metrics - Test RMSE: 3.7136
✅ Logged Holt-Winters model to MLflow
🏃 View run holt_winters_andaman and nicobar at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254/runs/cc59117c0e08459ba1c1b328eb09081c
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/970560080101903254

🔹 Determining best model...
Best model for Andaman And Nicobar: random_forest
Best model me

...

✅ Logged XGBoost model to MLflow: xgboost_uttarakhand
🏃 View run xgboost_uttarakhand at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/401887017500909504/runs/d887f534d8be4b3381e7f21a5d616ec7
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/401887017500909504

🔹 Training Holt-Winters model...
Holt-Winters Parameters: {'trend': 'add', 'seasonal': 'mul', 'seasonal_periods': 12}
Started MLflow run: b018a0591da7485f9ca4e112cca4e8ec
Holt-Winters metrics - Test RMSE: 6.7719
✅ Logged Holt-Winters model to MLflow
🏃 View run holt_winters_uttarakhand at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/401887017500909504/runs/b018a0591da7485f9ca4e112cca4e8ec
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/401887017500909504

🔹 Determining best model...
Best model for Uttarakhand: xgboost
Best model metrics:
   - train_rmse: 0.0997
   - te

2025/04/20 07:32:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'random_forest_west bengal' already exists. Creating a new version of this model...
2025/04/20 07:32:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_west bengal, version 2
Created version '2' of model 'random_forest_west bengal'.


✅ Logged Random Forest model to MLflow: random_forest_west bengal
🏃 View run random_forest_west bengal at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189/runs/e1d87e5d11d24690bd899aa92bc2126d
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189

🔹 Training XGBoost model...
XGBoost Parameters: {'n_estimators': 1000, 'learning_rate': 0.03, 'max_depth': 6, 'colsample_bytree': 0.8, 'subsample': 0.9, 'gamma': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.5, 'random_state': 42}
Started MLflow run: a8d7b5cf4af74ddf9381485866d8beb1
XGBoost metrics - Test RMSE: 1.9416, R²: 0.1770
Top 5 important features:
   - year: 0.3140
   - rolling_mean_3m: 0.2239
   - lag_1m: 0.1496
   - CPI: 0.1188
   - rolling_mean_6m: 0.1044


2025/04/20 07:32:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgboost_west bengal' already exists. Creating a new version of this model...
2025/04/20 07:32:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost_west bengal, version 2
Created version '2' of model 'xgboost_west bengal'.


✅ Logged XGBoost model to MLflow: xgboost_west bengal
🏃 View run xgboost_west bengal at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189/runs/a8d7b5cf4af74ddf9381485866d8beb1
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189

🔹 Training Holt-Winters model...
Holt-Winters Parameters: {'trend': 'add', 'seasonal': 'mul', 'seasonal_periods': 12}
Started MLflow run: af29c7ea7a5741028a3494330cf27c5c
Holt-Winters metrics - Test RMSE: 3.9181
✅ Logged Holt-Winters model to MLflow
🏃 View run holt_winters_west bengal at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189/runs/af29c7ea7a5741028a3494330cf27c5c
🧪 View experiment at: http://ec2-51-20-136-254.eu-north-1.compute.amazonaws.com:5000/#/experiments/708359278913264189

🔹 Determining best model...
Best model for West Bengal: random_forest
Best model metrics:
   - train_rmse: 0.3606
 

In [ ]:
def evaluate_wheat_price_models_minimal(forecaster_results, target_mape=20.0):
    """
    Minimal evaluation of wheat price forecasting models using only the metrics
    already calculated in the results dictionary.

    Args:
        forecaster_results: Results dictionary from WheatPriceForecaster.run_full_process()
        target_mape: Target MAPE value to achieve (default: 20.0%)

    Returns:
        Dictionary containing basic evaluation results
    """
    import numpy as np

    print("\n" + "="*80)
    print("📊 Minimal Model Evaluation")
    print("="*80)

    # Extract results
    all_state_models = forecaster_results['all_state_models']
    valid_models = {state: data for state, data in all_state_models.items() if data is not None}

    # Initialize results dictionary
    eval_results = {
        'mape_evaluation': {},
        'summary': {}
    }

    # 1. MAPE Target Evaluation
    print(f"\n🔹 Evaluating MAPE Target (<{target_mape}%)...")
    meet_target = []
    miss_target = []

    for state, model_info in valid_models.items():
        mape = model_info['metrics'].get('test_mape', float('inf'))
        if mape < target_mape:
            meet_target.append((state, mape))
        else:
            miss_target.append((state, mape))

    # Sort by MAPE (ascending)
    meet_target.sort(key=lambda x: x[1])
    miss_target.sort(key=lambda x: x[1])

    eval_results['mape_evaluation'] = {
        'target': target_mape,
        'meet_target': meet_target,
        'miss_target': miss_target,
        'meet_target_count': len(meet_target),
        'miss_target_count': len(miss_target),
        'success_rate': len(meet_target) / (len(meet_target) + len(miss_target)) * 100 if (len(meet_target) + len(miss_target)) > 0 else 0
    }

    print(f"States meeting MAPE target: {len(meet_target)}/{len(valid_models)} ({eval_results['mape_evaluation']['success_rate']:.1f}%)")
    if meet_target:
        print("Top 5 performing states:")
        for i, (state, mape) in enumerate(meet_target[:5]):
            print(f"  {i+1}. {state.title()}: MAPE = {mape:.2f}%")

    # 2. Generate summary
    print("\n🔹 Generating summary statistics...")

    # Calculate overall stats
    mape_values = [data['metrics'].get('test_mape') for data in valid_models.values()
                  if 'test_mape' in data['metrics']]
    rmse_values = [data['metrics'].get('test_rmse') for data in valid_models.values()
                  if 'test_rmse' in data['metrics']]
    r2_values = [data['metrics'].get('test_r2') for data in valid_models.values()
                if 'test_r2' in data['metrics'] and not np.isnan(data['metrics'].get('test_r2', float('nan')))]

    model_counts = {}
    for data in valid_models.values():
        model_type = data['best_model']
        model_counts[model_type] = model_counts.get(model_type, 0) + 1

    eval_results['summary'] = {
        'total_states': len(valid_models),
        'mean_mape': np.mean(mape_values) if mape_values else None,
        'median_mape': np.median(mape_values) if mape_values else None,
        'min_mape': np.min(mape_values) if mape_values else None,
        'max_mape': np.max(mape_values) if mape_values else None,
        'mean_rmse': np.mean(rmse_values) if rmse_values else None,
        'median_rmse': np.median(rmse_values) if rmse_values else None,
        'mean_r2': np.mean(r2_values) if r2_values else None,
        'model_distribution': model_counts
    }

    # Print summary
    print("\nOverall Performance Summary:")
    print(f"  Total states analyzed: {len(valid_models)}")
    if eval_results['summary']['mean_mape'] is not None:
        print(f"  Mean MAPE: {eval_results['summary']['mean_mape']:.2f}%")
        print(f"  Median MAPE: {eval_results['summary']['median_mape']:.2f}%")
        print(f"  MAPE range: {eval_results['summary']['min_mape']:.2f}% - {eval_results['summary']['max_mape']:.2f}%")
    if eval_results['summary']['mean_rmse'] is not None:
        print(f"  Mean RMSE: {eval_results['summary']['mean_rmse']:.4f}")
    if eval_results['summary']['mean_r2'] is not None:
        print(f"  Mean R²: {eval_results['summary']['mean_r2']:.4f}")

    print("\nModel Distribution:")
    for model_type, count in model_counts.items():
        print(f"  {model_type}: {count} states ({count/len(valid_models)*100:.1f}%)")

    print("\n" + "="*80)
    print("✅ Model evaluation completed")
    print("="*80)

    return eval_results

In [ ]:
evaluation_results = evaluate_wheat_price_models_minimal(results)

# You can now use evaluation_results for further analysis, reporting, or visualization
print(f"Number of states meeting MAPE target: {evaluation_results['mape_evaluation']['meet_target_count']}")


📊 Minimal Model Evaluation

🔹 Evaluating MAPE Target (<20.0%)...
States meeting MAPE target: 21/21 (100.0%)
Top 5 performing states:
  1. Tamil Nadu: MAPE = 2.64%
  2. Punjab: MAPE = 3.15%
  3. Delhi: MAPE = 3.19%
  4. Assam: MAPE = 3.26%
  5. Haryana: MAPE = 3.35%

🔹 Generating summary statistics...

Overall Performance Summary:
  Total states analyzed: 21
  Mean MAPE: 6.15%
  Median MAPE: 5.12%
  MAPE range: 2.64% - 17.43%
  Mean RMSE: 2.2765
  Mean R²: -1.6629

Model Distribution:
  random_forest: 7 states (33.3%)
  xgboost: 8 states (38.1%)
  holt_winters: 6 states (28.6%)

✅ Model evaluation completed
Number of states meeting MAPE target: 21
